In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
import tensorflow as tf

2022-10-24 22:14:21.057530: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-10-24 22:14:21.174712: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-10-24 22:14:21.174738: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-10-24 22:14:21.196003: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-10-24 22:14:21.943730: W tensorflow/stream_executor/platform/de

In [2]:
data =pd.read_feather('./tavbase/GS.feather')

In [3]:
#meia boca 
def deep_learning_dnn(df_dl, dep_var, classes):
    # Separa a variável dependente das demais
    deep_feat = df_dl.drop(columns=[dep_var], axis=1)
    deep_label = df_dl[dep_var]
    # Verifica os tipos das variáveis
    categorical_columns = [col for col in deep_feat.columns if len(deep_feat[col].unique()) == 2 or deep_feat[col].dtype == 'O']
    continuous_columns = [col for col in deep_feat.columns if len(deep_feat[col].unique()) > 2 and (deep_feat[col].dtype == 'int64' or deep_feat[col].dtype == 'float64')]
    # Verifica as colunas para normalização - as demais serão discretizadas - Função Bucketize do Tensor Flow
    cols_to_scale = continuous_columns[:]
    #cols_to_scale.remove('meses')
    # Ajusta as bases de treino e de teste
    XX_T = df_dl.drop(columns=[dep_var], axis=1)
    XX_t = df_dl.drop(columns=[dep_var], axis=1)
    yy_T = df_dl[dep_var]
    yy_t = df_dl[dep_var]
    # Normaliza as variáveis nas bases de treino e teste
    scaler = StandardScaler()
    XX_T.loc[:, cols_to_scale] = scaler.fit_transform(XX_T.loc[:, cols_to_scale])
    XX_t.loc[:, cols_to_scale] = scaler.fit_transform(XX_t.loc[:, cols_to_scale])
    # Ajustes das Variáveis Categórica - Não presentes neste modelo
    categorical_object_feat_cols = [tf.feature_column.embedding_column(
    tf.feature_column.categorical_column_with_hash_bucket(key=col, hash_bucket_size=1000), dimension=len(df_dl[col].unique()))
    for col in categorical_columns if df_dl[col].dtype == 'O']
    # Ajustes das Variáveis Categórica - Não presentes neste modelo
    categorical_integer_feat_cols = [tf.feature_column.embedding_column(
    tf.feature_column.categorical_column_with_identity(key=col, num_buckets=2), dimension=len(df_dl[col].unique()))
    for col in categorical_columns if df[col].dtype=='int64']
    continuous_feat_cols = [tf.feature_column.numeric_column(key=col) for col in continuous_columns] 
    feat_cols = categorical_object_feat_cols + \
                categorical_integer_feat_cols + \
                continuous_feat_cols
    # Rotina de DNN (Deep Neural Network)
    input_fun = tf.compat.v1.estimator.inputs.pandas_input_fn(XX_T, yy_T, batch_size=50, num_epochs=1000, shuffle=True)
    pred_input_fun = tf.compat.v1.estimator.inputs.pandas_input_fn(XX_t, batch_size=50, shuffle=False)
    DNN_model = tf.estimator.DNNClassifier(hidden_units=[10, 10, 10], feature_columns=feat_cols, n_classes=classes)
    DNN_model.train(input_fn=input_fun, steps=5000)
    # Resgata os resultados da DNN
    predictions = DNN_model.predict(pred_input_fun)
    pred = list(predictions)
    return pred

In [5]:
probabilidade = deep_learning_dnn(grupo, 'Benefit', 2)
probabilidade_classe = []
for i in range(len(probabilidade)):
    probabilidade_classe.append(probabilidade[i]["class_ids"][0])
probabilidade_prob0 = []
for i in range(len(probabilidade)):
    probabilidade_prob0.append(probabilidade[i]["probabilities"][0])
probabilidade_prob1 = []
for i in range(len(probabilidade)):
    probabilidade_prob1.append(probabilidade[i]["probabilities"][1]) 
grupo['dl_classe'] = probabilidade_classe
grupo['lucro_0'] = probabilidade_prob0
grupo['lucro_1'] = probabilidade_prob1
grupo = grupo.reset_index()
grupo

grupo.to_feather('probabilidade_pais.feather')

NameError: name 'grupo' is not defined